In [2]:
pip install beautifulsoup4

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 77, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_

In [9]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def is_valid_pdf_url(url):
    try:
        response = requests.head(url, headers={'User-Agent': 'Mozilla/5.0'}, allow_redirects=True)
        content_type = response.headers.get('Content-Type', '')
        return 'application/pdf' in content_type
    except requests.RequestException:
        return False

def extract_actual_url(href):
    if href.startswith('/url?'):
        href = urllib.parse.parse_qs(urllib.parse.urlparse(href).query).get('url', [None])[0]
    return href

def get_annual_report_link(company_name, year):
    search_query = f"{company_name} {year} annual report"
    url = f"https://www.google.com/search?q={search_query}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)

    # Check if the response is HTML
    if 'text/html' not in response.headers.get('Content-Type', ''):
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract and validate links containing 'annual report'
    for link in soup.find_all('a'):
        href = link.get('href')
        if 'annual report' in link.text.lower() and href:
            href = extract_actual_url(href)
            if href and is_valid_pdf_url(href):
                return href  # Return the first valid PDF link
            else:
                # Try to check if the link is a valid HTML page containing the PDF link
                if href and 'http' in href:
                    sub_response = requests.get(href, headers=headers)
                    if 'text/html' in sub_response.headers.get('Content-Type', ''):
                        sub_soup = BeautifulSoup(sub_response.text, 'html.parser')
                        for sub_link in sub_soup.find_all('a'):
                            sub_href = sub_link.get('href')
                            sub_href = extract_actual_url(sub_href)
                            if sub_href and is_valid_pdf_url(sub_href):
                                return sub_href

    return None

def get_annual_report_links(companies, year):
    links = {}
    for company in companies:
        link = get_annual_report_link(company, year)
        if link:
            links[company] = link
    return links

# Example usage

companies = [
    "3i",
    "Admiral Group",
    "Airtel Africa",
    "Anglo American plc",
    "Antofagasta plc",
    "Ashtead Group",
    "Associated British Foods",
    "AstraZeneca",
    "Auto Trader Group",
    "Aviva",
    "B&M",
    "BAE Systems",
    "Barclays",
    "Barratt Developments",
    "Beazley",
    "Berkeley Group Holdings",
    "BP",
    "British American Tobacco",
    "BT Group",
    "Bunzl",
    "Burberry",
    "Centrica",
    "Coca-Cola HBC",
    "Compass Group",
    "Convatec",
    "Croda International",
    "DCC plc",
    "Diageo",
    "Diploma",
    "Entain",
    "EasyJet",
    "Experian",
    "F & C Investment Trust",
    "Frasers Group",
    "Fresnillo plc",
    "Glencore",
    "GSK plc",
    "Haleon",
    "Halma plc",
    "Hargreaves Lansdown",
    "Hikma Pharmaceuticals",
    "Howdens Joinery",
    "HSBC",
    "IHG Hotels & Resorts",
    "IMI",
    "Imperial Brands",
    "Informa",
    "Intermediate Capital Group",
    "International Airlines Group",
    "Intertek",
    "JD Sports",
    "Kingfisher plc",
    "Land Securities",
    "Legal & General",
    "Lloyds Banking Group",
    "London Stock Exchange Group",
    "M&G",
    "Marks & Spencer",
    "Melrose Industries",
    "Mondi",
    "National Grid plc",
    "NatWest Group",
    "Next plc",
    "Ocado Group",
    "Pearson plc",
    "Pershing Square Holdings",
    "Persimmon",
    "Phoenix Group",
    "Prudential plc",
    "Reckitt",
    "RELX",
    "Rentokil Initial",
    "Rightmove",
    "Rio Tinto",
    "Rolls-Royce Holdings",
    "RS Group plc",
    "Sage Group",
    "Sainsbury's",
    "Schroders",
    "Scottish Mortgage Investment Trust",
    "Segro",
    "Severn Trent",
    "Shell plc",
    "DS Smith",
    "Smiths Group",
    "Smith & Nephew",
    "Smurfit Kappa",
    "Spirax-Sarco Engineering",
    "SSE plc",
    "Standard Chartered",
    "St. James's Place plc",
    "Taylor Wimpey",
    "Tesco",
    "Unilever",
    "United Utilities",
    "Unite Group",
    "Vodafone Group",
    "Weir Group",
    "Whitbread",
    "WPP plc",
    "3i Infrastructure",
    "4imprint",
    "Abrdn",
    "abrdn Private Equity Opportunities Trust",
    "Alliance Trust",
    "Allianz Technology Trust",
    "AO World",
    "Apax Global Alpha",
    "Ascential",
    "Ashmore Group",
    "Asia Dragon Trust",
    "Assura",
    "Aston Martin Lagonda",
    "Auction Technology Group",
    "AVI Global Trust",
    "Babcock International",
    "Baillie Gifford Japan Trust",
    "Baillie Gifford US Growth Trust",
    "Bakkavör",
    "Balanced Commercial Property Trust",
    "Balfour Beatty",
    "Baltic Classifieds",
    "Bankers Investment Trust",
    "Bank of Georgia",
    "A.G. Barr",
    "BBGI",
    "AJ Bell",
    "Bellevue Healthcare Trust",
    "Bellway",
    "BH Macro",
    "Big Yellow Group",
    "BlackRock Greater Europe Investment Trust",
    "BlackRock Smaller Companies Trust",
    "BlackRock Throgmorton Trust",
    "BlackRock World Mining Trust",
    "Bluefield Solar Income Fund",
    "Bodycote",
    "Breedon Group",
    "Bridgepoint Group",
    "British Land",
    "Britvic",
    "Brunner Investment Trust",
    "Bytes Technology Group",
    "C&C Group",
    "Caledonia Investments",
    "Capital Gearing Trust",
    "Carnival Corporation & plc",
    "Centamin",
    "Chemring Group",
    "City of London Investment Trust",
    "Clarkson",
    "Close Brothers Group",
    "CMC Markets",
    "Coats Group",
    "Computacenter",
    "Cranswick",
    "Crest Nicholson",
    "Currys",
    "Darktrace",
    "Derwent London",
    "Direct Line",
    "discoverIE",
    "Diversified Energy",
    "Domino's Pizza",
    "Dowlais Group",
    "Drax Group",
    "Dr. Martens",
    "Dunelm Group",
    "Edinburgh Investment Trust",
    "Edinburgh Worldwide Investment Trust",
    "Elementis",
    "Empiric Student Property",
    "Endeavour Mining",
    "Energean",
    "Essentra",
    "European Opportunities Trust",
    "European Smaller Companies Trust",
    "Ferrexpo",
    "Fidelity China Special Situations",
    "Fidelity Emerging Markets",
    "Fidelity European Trust",
    "Fidelity Special Values",
    "Finsbury Growth & Income Trust",
    "FirstGroup",
    "Foresight Group",
    "Foresight Solar Fund",
    "Future",
    "Games Workshop Group",
    "GCP Infrastructure Investments",
    "Genuit Group",
    "Genus",
    "Global Smaller Companies Trust",
    "Grafton Group",
    "Grainger",
    "Great Portland Estates",
    "Greencoat UK Wind",
    "Greencore",
    "Greggs",
    "Hammerson",
    "Harbour Energy",
    "HarbourVest Global Private Equity",
    "Hays",
    "Helios Towers",
    "Henderson Smaller Companies Investment Trust",
    "Herald Investment Trust",
    "Hg Capital Trust",
    "HICL Infrastructure Company",
    "Hill & Smith",
    "Hilton Food Group",
    "Hipgnosis Songs Fund",
    "Hiscox",
    "Hochschild Mining",
    "Hollywood Bowl Group",
    "Hunting",
    "Ibstock",
    "ICG Enterprise Trust",
    "IG Group Holdings",
    "Impax Environmental Markets",
    "Inchcape",
    "Indivior",
    "IntegraFin Holdings",
    "International Distributions Services",
    "International Public Partnerships",
    "Investec",
    "IP Group",
    "Ithaca Energy",
    "ITV",
    "IWG",
    "JLEN Environmental Assets Group",
    "Johnson Matthey",
    "JPMorgan American Investment Trust",
    "JPMorgan Emerging Markets Investment Trust",
    "JPMorgan European Discovery",
    "JPMorgan Global Growth & Income",
    "JPMorgan Indian Investment Trust",
    "JPMorgan Japanese Investment Trust",
    "JTC",
    "Jupiter Fund Management",
    "Just Group",
    "Kainos",
    "Keller Group",
    "Kier Group",
    "Lancashire Holdings",
    "Law Debenture",
    "LondonMetric Property",
    "Man Group",
    "Marshalls",
    "ME Group International",
    "Mercantile Investment Trust",
    "Merchants Trust",
    "Mitchells & Butlers",
    "Mitie",
    "Mobico Group",
    "Molten Ventures",
    "Moneysupermarket.com Group",
    "Monks Investment Trust",
    "Moonpig",
    "Morgan Advanced Materials",
    "Morgan Sindall Group",
    "Murray Income Trust",
    "Murray International Trust",
    "NB Private Equity Partners",
    "Network International",
    "NextEnergy Solar Fund",
    "Ninety One",
    "North Atlantic Smaller Companies Investment Trust",
    "Octopus Renewables Infrastructure Trust",
    "OneSavings Bank",
    "Oxford Instruments",
    "Pacific Horizon Investment Trust",
    "PageGroup",
    "Pantheon International",
    "Paragon Banking Group",
    "Patria Private Equity Trust",
    "Pennon Group",
    "Personal Assets Trust",
    "Petershill Partners",
    "Pets at Home",
    "Playtech",
    "Plus500",
    "Polar Capital Technology Trust",
    "PPHE Hotel Group",
    "Premier Foods",
    "Primary Health Properties",
    "PureTech Health",
    "PZ Cussons",
    "Qinetiq",
    "Quilter",
    "Rathbones",
    "Redrow",
    "Renishaw",
    "RHI Magnesita",
    "RIT Capital Partners",
    "Rotork",
    "Ruffer Investment Company",
    "Safestore",
    "Savills",
    "Schroder AsiaPacific Fund",
    "Schroder Oriental Income Fund",
    "Scottish American Investment Company",
    "SDCL Energy Efficiency Income Trust",
    "Senior",
    "Sequoia Economic Infrastructure Income Fund",
    "Serco",
    "Shaftesbury Capital",
    "Sirius Real Estate",
    "Smithson Investment Trust",
    "Softcat",
    "Spectris",
    "Spire Healthcare",
    "Spirent",
    "SSP Group",
    "SThree",
    "Supermarket Income REIT",
    "Syncona",
    "Target Healthcare REIT",
    "TBC Bank",
    "Telecom Plus",
    "Temple Bar Investment Trust",
    "Templeton Emerging Markets Investment Trust",
    "The Renewables Infrastructure Group",
    "TI Fluid Systems",
    "TP ICAP",
    "Trainline",
    "Travis Perkins",
    "Tritax Big Box REIT",
    "Tritax EuroBox",
    "TR Property Investment Trust",
    "Trustpilot",
    "Twentyfour Income Fund",
    "Tyman",
    "Urban Logistics REIT",
    "Vesuvius",
    "Victrex",
    "Vietnam Enterprise Investments",
    "VinaCapital Vietnam Opportunity Fund",
    "Virgin Money UK",
    "Vistry Group",
    "Volution Group",
    "W. A. G. Payment Solutions",
    "Watches of Switzerland",
    "Wetherspoon (J D)",
    "W H Smith",
    "Witan Investment Trust",
    "Wizz Air",
    "Wood Group",
    "Workspace Group",
    "Worldwide Healthcare Trust",
    "Zigup"
]

year = 2023
report_links = get_annual_report_links(companies, year)

for company, link in report_links.items():
    print(f"Annual Report Link for {company} {year}: {link}")


KeyboardInterrupt: 

ignore below

In [18]:
#For single entries
import requests
from bs4 import BeautifulSoup
import urllib.parse

def is_valid_pdf_url(url):
    try:
        response = requests.head(url, headers={'User-Agent': 'Mozilla/5.0'}, allow_redirects=True)
        content_type = response.headers.get('Content-Type', '')
        return 'application/pdf' in content_type
    except requests.RequestException as e:
        print(f"Error checking URL: {e}")
        return False

def extract_actual_url(href):
    if href.startswith('/url?'):
        href = urllib.parse.parse_qs(urllib.parse.urlparse(href).query).get('url', [None])[0]
    return href

def get_annual_report_link(company_name, year):
    search_query = f"{company_name} {year} annual report"
    url = f"https://www.google.com/search?q={search_query}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)

    # Check if the response is HTML
    if 'text/html' not in response.headers.get('Content-Type', ''):
        print("Unexpected content type:", response.headers.get('Content-Type'))
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract and validate links containing 'annual report'
    for link in soup.find_all('a'):
        href = link.get('href')
        if 'annual report' in link.text.lower() and href:
            href = extract_actual_url(href)
            if href and is_valid_pdf_url(href):
                return href  # Return the first valid PDF link
            else:
                # Try to check if the link is a valid HTML page containing the PDF link
                if href and 'http' in href:
                    sub_response = requests.get(href, headers=headers)
                    if 'text/html' in sub_response.headers.get('Content-Type', ''):
                        sub_soup = BeautifulSoup(sub_response.text, 'html.parser')
                        for sub_link in sub_soup.find_all('a'):
                            sub_href = sub_link.get('href')
                            sub_href = extract_actual_url(sub_href)
                            if sub_href and is_valid_pdf_url(sub_href):
                                return sub_href

    return None

# Example usage
company_name = "Tesco"
year = 2023
link = get_annual_report_link(company_name, year)
if link:
    print(f"Annual Report Link for {company_name} {year}: {link}")
else:
    print(f"No valid annual report link found for {company_name} {year}")



KeyboardInterrupt: 